Test

In [ ]:
!wget "https://raw.githubusercontent.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/master/src/datasets/exploration_datasets/gold/qa_dataset_col_mex_news_squad2.json" -O "qa_dataset_col_mex_news_squad2.json"

--2025-10-24 02:02:14--  https://raw.githubusercontent.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/master/src/datasets/exploration_datasets/gold/qa_dataset_col_mex_news_squad2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133 [text/plain]
Saving to: ‘qa_dataset_col_mex_news_squad2.json’

qa_dataset_col_mex_ 100%[===================>]     133  --.-KB/s    in 0s      

2025-10-24 02:02:14 (3.06 MB/s) - ‘qa_dataset_col_mex_news_squad2.json’ saved [133/133]



In [ ]:
!ls -lh "/content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json"
!head -n 10 "/content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json"

-rw------- 1 root root 133 Oct 24 02:01 /content/drive/MyDrive/qa_dataset_col_mex_news_squad2.json
version https://git-lfs.github.com/spec/v1
oid sha256:7103b65b3b45ee4d63d57681fceae6df1123a839f483467f74a63583f3b6e14e
size 20171386


In [2]:
!pip install transformers evaluate torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00


In [13]:
!pip install simpletransformers transformers datasets huggingface_hub scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.8/330.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=b0a3a045fb378c2d50f4abe2b9f8da51f84e08b566dd2adb1fa95d1a86a54286
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [14]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import evaluate
import json
from tqdm import tqdm
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from sklearn.model_selection import train_test_split

In [29]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [4]:
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

GPU available: True
GPU name: Tesla T4


# Paso 1: Train directly with your data

In [20]:
import json

with open("/content/qa_dataset_col_mex_news_squad2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

print(f"✅ Loaded {len(dataset)} records successfully!")


✅ Loaded 1 records successfully!


In [21]:
dataset = dataset["data"]

In [34]:
# 🔧 1️⃣ Flatten your dataset so each row has 'context' and 'qas'
def flatten_squad(dataset):
    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data

flattened_data = flatten_squad(dataset)

# 🔧 2️⃣ Split into train and eval sets (70/30)
train_data, eval_data = train_test_split(flattened_data, test_size=0.3, random_state=42)

In [38]:
print(f"✅ Training samples: {len(train_data)}")
print(f"✅ Eval samples: {len(eval_data)}")

✅ Training samples: 2273
✅ Eval samples: 975


# Paso 2: Define hyperparameters

In [35]:
model_args = QuestionAnsweringArgs()

# Training behavior
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 4
model_args.learning_rate = 2e-5
model_args.gradient_accumulation_steps = 1
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.save_steps = -1
model_args.best_model_dir = "./outputs/best_model/"
model_args.output_dir = "./outputs/"

# Optimization
model_args.max_seq_length = 384
model_args.doc_stride = 128
model_args.warmup_ratio = 0.1
model_args.max_answer_length = 30

# Logging
model_args.logging_steps = 100
model_args.evaluate_during_training_verbose = True
model_args.manual_seed = 42

# Resource handling
model_args.use_multiprocessing = False  # safer for notebooks
model_args.fp16 = torch.cuda.is_available()  # use mixed precision if CUDA available

# Paso 3: Load model

In [36]:
model = QuestionAnsweringModel(
    model_type="bert",
    model_name="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",  # BETO distilled
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

The following layers were not sharded: bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.attention.self.value.bias, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.intermediate.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.attention.output.LayerNorm.weight, qa_outputs.weight, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.attention.self.value.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.position_embeddings.weight, qa_outputs.bias, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.embeddings.LayerNorm.bias, bert.embeddings.LayerNorm.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.key.bias, bert.encoder.layer.*.attention.self.query.bias, bert.encoder.layer.*.attention.self.query.weight, ber

# Paso 4: TRAIN THE MODEL

In [37]:
model.train_model(train_data, eval_data=eval_data)

  5%|▍         | 661/14684 [00:19<06:53, 33.92it/s]


KeyboardInterrupt: 

# Paso 5: EVALUATE

In [ ]:
result, texts = model.eval_model(eval_data)
print("📊 Evaluation results:")
print(result)

# Paso 6:

In [ ]:
import os

os.makedirs("./outputs/final_model/", exist_ok=True)
model.save_model("./outputs/final_model/")
print("✅ Model saved at ./outputs/final_model/")


In [ ]:
from google.colab import files
!zip -r final_model.zip ./outputs/final_model/
files.download("final_model.zip")

In [9]:
#model_name = "mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
model_name = "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"

qa_pipeline = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=model_name
)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
contexto = """
Gabriel García Márquez fue un novelista colombiano, ganador del Premio Nobel de Literatura en 1982.
Es conocido por obras como 'Cien años de soledad' y 'El amor en los tiempos del cólera'.
"""

pregunta = "¿Quién ganó el Premio Nobel de Literatura en 1982?"

resultado = qa_pipeline({
    "context": contexto,
    "question": pregunta
})

print(resultado)

{'score': 0.9416845025261864, 'start': 1, 'end': 23, 'answer': 'Gabriel García Márquez'}


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [ ]:
from tqdm import tqdm

predictions = []
references = []

for article in tqdm(dataset["data"]):
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]
        for qa in paragraph["qas"]:
            question = qa["question"]

            # Model prediction
            pred = qa_pipeline(question=question, context=context)

            predictions.append({
                "id": qa["id"],
                "prediction_text": pred.get("answer", ""),
                "no_answer_probability": 1 - pred.get("score", 0)
            })

            # Handle multiple gold answers if present
            gold_texts = [ans["text"] for ans in qa["answers"]]
            gold_starts = [ans.get("answer_start", 0) for ans in qa["answers"]]

            # ✅ FIX: include both "text" and "answer_start"
            references.append({
                "id": qa["id"],
                "answers": {
                    "text": gold_texts,
                    "answer_start": gold_starts
                }
            })



100%|██████████| 3248/3248 [13:49<00:00,  3.91it/s]


In [ ]:
metric = evaluate.load("squad_v2")

In [ ]:
references

[{'id': 'qa-17811', 'answers': {'text': ['Illinois'], 'answer_start': [153]}},
 {'id': 'qa-17812',
  'answers': {'text': ['Tuxedomoon'], 'answer_start': [278]}},
 {'id': 'qa-17813',
  'answers': {'text': ['Qué Viva México'], 'answer_start': [2502]}},
 {'id': 'qa-17814', 'answers': {'text': ['1993'], 'answer_start': [626]}},
 {'id': 'qa-17815',
  'answers': {'text': ['fiesta de la Candelaria'], 'answer_start': [2698]}},
 {'id': 'qa-17816', 'answers': {'text': [], 'answer_start': []}},
 {'id': 'qa-17817', 'answers': {'text': [], 'answer_start': []}},
 {'id': 'qa-17778',
  'answers': {'text': ['La funcionaria'], 'answer_start': [2]}},
 {'id': 'qa-17779',
  'answers': {'text': ['19 por ciento'], 'answer_start': [478]}},
 {'id': 'qa-17781',
  'answers': {'text': ['73.7 por ciento'], 'answer_start': [1730]}},
 {'id': 'qa-17782', 'answers': {'text': [], 'answer_start': []}},
 {'id': 'qa-17938',
  'answers': {'text': ['Chimalistac'], 'answer_start': [363]}},
 {'id': 'qa-17939',
  'answers': {'

In [ ]:
results = metric.compute(predictions=predictions, references=references)

In [ ]:
metric = evaluate.load("squad_v2")
results = metric.compute(predictions=predictions, references=references)
print(results)

{'exact': 40.33673566727082, 'f1': 52.04827644347888, 'total': 20966, 'HasAns_exact': 58.15441783649876, 'HasAns_f1': 75.0512086370753, 'HasAns_total': 14532, 'NoAns_exact': 0.09325458501709667, 'NoAns_f1': 0.09325458501709667, 'NoAns_total': 6434, 'best_exact': 55.33244300295717, 'best_exact_thresh': 0.6415619850158691, 'best_f1': 64.63131456858878, 'best_f1_thresh': 0.7467862963676453}


In [ ]:
print("📊 Evaluation Results:")
print(f"Exact Match (EM): {results['exact']:.2f}%")
print(f"F1 Score: {results['f1']:.2f}%")
print(f"Total Examples: {results['total']}")
print(f"Has Answer (EM): {results.get('HasAns_exact', 0):.2f}%")
print(f"No Answer (EM): {results.get('NoAns_exact', 0):.2f}%")


📊 Evaluation Results:
Exact Match (EM): 40.34%
F1 Score: 52.05%
Total Examples: 20966
Has Answer (EM): 58.15%
No Answer (EM): 0.09%
